In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import time
import collections

In [ ]:
%%time
data=pd.read_excel("IT Incidents Dashboard FY22 Q4 V1.xlsx")


Considering only closed ticket to proceed further for analysis, by assuming the closed tickets are having correct values

In [ ]:
closed_ticket=data[data['Incident state']=='Closed']
closed_ticket.reset_index(drop=True,inplace=True)

In [ ]:
closed_ticket= closed_ticket[['Number','Name','Opened','Assignment group', 'Priority', 'Urgency', 'Configuration item', 'Subcategory','Category','Resolved','Business resolve time']]


In [ ]:
closed_ticket.isnull().sum()

In [ ]:
closed_ticket.dropna(subset=['Name'],inplace=True)
closed_ticket.reset_index(drop=True,inplace=True)

In [ ]:
closed_ticket.isnull().sum()

## String validation for name column

In [ ]:
def string_validation(text):
    
    #pattern for identifying the mail id
    regex1 = re.compile(r'([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+|([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+ ')
    
    
    #pattern for identifying the url link
    regex2 = re.compile(r'(https?://[^\s]+)|(www.[^\s]+)|(http?://[^\s]+)|(https?://[^\s]+)+\ |(www.[^\s]+\ )|(http?://[^\s]+\ )')
    
    
    try:
        
        # checking the string containing onlr mail id
        if re.fullmatch(regex1, text):
            return True
        
       
        
        #checking the string is having any http link
        elif re.fullmatch(regex2, text):
            return True
        
    
        else:
            
            if text.isdigit():
                return True
            
            else:

                #removing the some special character and sign
                text=re.sub(r'\+|\-|\\|\.|\(|\)|\/','', text)

                #removing the space between digits
                text=re.sub(r'(\d)\s+(\d)', r'\1\2', text)

                #checking the string contains only numbers or not
                if text.isdigit():
                    return True
                
                else:
                    return False
                
        
    except Exception as e:
        return e

In [ ]:
%%time
closed_ticket["name_string_validation"]=closed_ticket["Name"].apply(string_validation)

In [ ]:
closed_ticket.name_string_validation.value_counts()

In [ ]:
closed_ticket.drop(closed_ticket[closed_ticket['name_string_validation'] == True].index,inplace=True)

In [ ]:
closed_ticket.reset_index(drop=True,inplace=True)

## Language detection and transalation to english

In [ ]:
from langdetect import detect
from deep_translator import GoogleTranslator

In [ ]:
def lang_detect(data):                                        
   try:                                                          
      return detect(data)                                      
   except:                                                       
      return 'Not identified'  

In [ ]:
%%time
closed_ticket['langue'] = closed_ticket['Name'].apply(lang_detect)

In [ ]:
closed_ticket.langue.value_counts()

In [ ]:
closed_ticket = closed_ticket[closed_ticket['langue'] != 'Not identified']
closed_ticket.reset_index(drop=True,inplace=True)

In [ ]:
def lang_translate(text,lang):
  try:
    if lang == 'en':
        return text
    
    else:
        return  GoogleTranslator(source='auto', target='en').translate(text)
  except:
    return text

In [ ]:
%%time
closed_ticket['translated_text'] = closed_ticket.apply(lambda x: lang_translate(x['Name'], x['langue']), axis=1)

## Text pre-processing & Identifying the non-english character

In [ ]:
import nltk
import string
stopwords = nltk.corpus.stopwords.words('english')

In [ ]:
## to clean the text data

def text_preprocessing(text,language):
    
    #removing the some special character and sign
    text=re.sub(r'\’|\【|\】|\“|\”|\–|\…|\´|\‐|\’|\—|\•|\‘|\°|\€|\™|\uf0e0|\®|\，|\|\！','', text) 
    
    #removing the some special character and sign
    text=text.translate(str.maketrans('','',string.punctuation))
    
    #will replace newline with space
    text = re.sub("\n"," ",text)
    
    #will convert to lower case
    text = text.lower()
    
    # removing the stop words
    text = ' '.join([word for word in text.split() if word not in (stopwords)])
    
    #checking for the non-english characters
    join_text=re.sub(r'\ |\  ','', text)
    join_text=join_text.strip()
    if join_text.isalnum ():
        return text
    else:
        #checking='Non english'
        if language == 'en':
            return text
        else:
            return 'Non english'

In [ ]:
%%time
closed_ticket['cleaned_name'] = closed_ticket.apply(lambda x: text_preprocessing(x['translated_text'],x['langue']), axis=1)

In [ ]:
non_english_character=closed_ticket[closed_ticket['cleaned_name']=='Non english']
non_english_character.reset_index(drop=True,inplace=True)
non_english_character.shape

In [ ]:
english_character=closed_ticket[closed_ticket['cleaned_name']!='Non english']
english_character.reset_index(drop=True,inplace=True)
english_character.shape

In [ ]:
%%time
non_english_character['translated_text'] = non_english_character.apply(lambda x: lang_translate(x['translated_text'], x['langue']), axis=1)

In [ ]:
%%time
non_english_character['cleaned_name'] = non_english_character.apply(lambda x: text_preprocessing(x['translated_text'],x['langue']), axis=1)

In [ ]:
print(non_english_character.shape)
print(english_character.shape)

In [ ]:
closed_ticket=pd.concat([english_character, non_english_character])
closed_ticket.reset_index(drop=True,inplace=True)

In [ ]:
def checking_len_string(text):
    
    string_length = len(text.split())
    character_length = len(text)
                        
    if string_length <= 1 or character_length <= 1:
        return "Not Valid"
                        
    else:
        return "Valid"

In [ ]:
%%time
closed_ticket['string_length'] = closed_ticket['cleaned_name'].apply(checking_len_string)

In [ ]:
closed_ticket['string_length'].value_counts()

In [ ]:
closed_ticket.drop(closed_ticket[closed_ticket['string_length'] == "Not Valid"].index,inplace=True)

In [ ]:
closed_ticket.shape

In [ ]:
closed_ticket.to_csv('closed_ticket.csv',index=False)

## Priortize ticket (keyword identification based)

In [ ]:
closed_ticket.Priority.value_counts()

In [ ]:
data=closed_ticket[['Number','translated_text','cleaned_name','Priority','Urgency']]

In [ ]:
data['cleaned_name']=data['cleaned_name'].astype(str)
data['translated_text']=data['translated_text'].astype(str)

In [ ]:
def new_priority(df):

    
    if df['Priority'] == '2 - Medium':
        if df['cleaned_name'].startswith(('p1', 'critical','escalating','escalate')):
            new_priority = '1 - High'
            return new_priority
        else:
            return df['Priority']
    
    elif df['Priority'] == '3 - Low':
        medium_prority_keyword = re.compile(r'failure notification|host alert high|tablespace|load failed|workflow failed|urgent action')
        if medium_prority_keyword.findall(df['cleaned_name']):
            new_priority = '2 - Medium'
            return new_priority
        else:
            return df['Priority'] 
    else:
        return df['Priority']
        
    

In [ ]:
%%time
data['new_priority'] = data.apply(new_priority,axis=1)

In [ ]:
data.Priority.value_counts()

In [ ]:
data.new_priority.value_counts()

In [ ]:
def Validation(df):
    
    if df['Priority'] == df['new_priority']:
        return "Same"
    else:
        return "Not same"

In [ ]:
%%time
data['Validation'] = data.apply(Validation,axis=1)

In [ ]:
data['Validation'].value_counts()

## Keyword extraction

In [ ]:
from rake_nltk import Rake
rake_nltk_var = Rake()

In [ ]:
def keyword_extraction(text):
    
    try:
        #extracting the keyword from the document
        rake_nltk_var.extract_keywords_from_text(text)
        keyword_extracted = rake_nltk_var.get_ranked_phrases()
        if len(keyword_extracted) !=0:
            keyword_extracted=str(keyword_extracted).replace('[','').replace(']','')
            return keyword_extracted
        else:
            return 'None'
        
    
    except Exception as e:
        return e

In [ ]:
%%time
data['keyword'] = data['translated_text'].apply(keyword_extraction)

In [ ]:
data.drop(data[data['keyword'] == 'None'].index,inplace=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Create a Vectorizer Object
vectorizer = CountVectorizer()
 
vectorizer.fit(data['keyword'])

keyword_count = vectorizer.vocabulary_
  

In [ ]:
keyword_df=pd.DataFrame.from_dict(keyword_count, orient='index',).reset_index()
keyword_df.columns = ['word', 'count']

In [ ]:
keyword_df

In [ ]:
def is_numeric(text):
    
    if text.isnumeric ():
        return 'yes'
    else:
        return 'no'

In [ ]:
keyword_df['is_numeric']=keyword_df['word'].apply(is_numeric)
keyword_df['is_numeric'].value_counts()

In [ ]:
keyword_df.drop(keyword_df[keyword_df['is_numeric'] == 'yes'].index,inplace=True)

In [ ]:
keyword_df.drop(columns=['is_numeric'],inplace=True)
keyword_df.reset_index(drop=True,inplace=True)

In [ ]:
keyword_df.to_csv('keywords.csv',index=False)

In [ ]:
keyword_df

In [ ]:
word_count = ' '.join(i for i in data['cleaned_name'])
filtered_words = word_count.split()
counted_words = collections.Counter(filtered_words)

In [ ]:
words=dict(counted_words)
type(words)

In [ ]:
keyword_count=pd.DataFrame.from_dict(words, orient='index',).reset_index()
keyword_count.columns = ['word', 'count']

In [ ]:
keyword_count['is_numeric']=keyword_count['word'].apply(is_numeric)
keyword_count['is_numeric'].value_counts()

In [ ]:
keyword_count.drop(keyword_count[keyword_count['is_numeric'] == 'yes'].index,inplace=True)

In [ ]:
keyword_count.drop(columns=['is_numeric'],inplace=True)
keyword_count.reset_index(drop=True,inplace=True)

In [ ]:
keyword_count.to_csv('keywordsList.csv',index=False)

In [ ]:
keyword_count

In [ ]:
data_new=data.copy()

In [ ]:
data_low=data_new[data_new['new_priority']=='3 - Low']

In [ ]:
word_count = ' '.join(i for i in data_low['cleaned_name'])
filtered_words = word_count.split()
counted_words = collections.Counter(filtered_words)
words=dict(counted_words)

In [ ]:
keyword_count=pd.DataFrame.from_dict(words, orient='index',).reset_index()
keyword_count.columns = ['word', 'count']
keyword_count['is_numeric']=keyword_count['word'].apply(is_numeric)
keyword_count['is_numeric'].value_counts()

In [ ]:
keyword_count.drop(keyword_count[keyword_count['is_numeric'] == 'yes'].index,inplace=True)
keyword_count.drop(columns=['is_numeric'],inplace=True)
keyword_count.reset_index(drop=True,inplace=True)

In [ ]:
keyword_count.to_csv('low_priority_keyword.csv',index=False)

In [ ]:
keyword_count.shape

In [ ]:
data_medium=data_new[data_new['new_priority']=='2 - Medium']

In [ ]:
word_count = ' '.join(i for i in data_medium['cleaned_name'])
filtered_words = word_count.split()
counted_words = collections.Counter(filtered_words)
words=dict(counted_words)

In [ ]:
keyword_count=pd.DataFrame.from_dict(words, orient='index',).reset_index()
keyword_count.columns = ['word', 'count']
keyword_count['is_numeric']=keyword_count['word'].apply(is_numeric)
keyword_count['is_numeric'].value_counts()

In [ ]:
keyword_count.drop(keyword_count[keyword_count['is_numeric'] == 'yes'].index,inplace=True)
keyword_count.drop(columns=['is_numeric'],inplace=True)
keyword_count.reset_index(drop=True,inplace=True)

In [ ]:
keyword_count.to_csv('medium_priority_keyword.csv',index=False)

In [ ]:
keyword_count

In [ ]:
data_high=data_new[data_new['new_priority']=='1 - High']

In [ ]:
word_count = ' '.join(i for i in data_high['cleaned_name'])
filtered_words = word_count.split()
counted_words = collections.Counter(filtered_words)
words=dict(counted_words)

In [ ]:
keyword_count=pd.DataFrame.from_dict(words, orient='index',).reset_index()
keyword_count.columns = ['word', 'count']
keyword_count['is_numeric']=keyword_count['word'].apply(is_numeric)
keyword_count['is_numeric'].value_counts()

In [ ]:
keyword_count.drop(keyword_count[keyword_count['is_numeric'] == 'yes'].index,inplace=True)
keyword_count.drop(columns=['is_numeric'],inplace=True)
keyword_count.reset_index(drop=True,inplace=True)

In [ ]:
keyword_count.to_csv('high_priority_keyword.csv',index=False)

In [ ]:
keyword_count

In [ ]:
data_new[data_new['new_priority']=='1 - High']